# SkyCast: Know before you go

This notebook covers comprehensive data cleaning, feature engineering, integrity checks, summary statistics, pattern identification, outlier handling, and visualizations for the flight delay dataset. The dataset used is the US DOT flight delays dataset from Kaggle.

In [ ]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from mpl_toolkits.basemap import Basemap
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
%matplotlib inline

## 1. Data Loading & Overview

Load the flights and airports datasets. Show variable types, missing values, and missing value percentages.

In [ ]:
flights = pd.read_csv('flights.csv', low_memory=False)
airports = pd.read_csv('airports.csv')
print('Flights shape:', flights.shape)
tab_info = pd.DataFrame(flights.dtypes).T.rename(index={0:'column type'})
tab_info = tab_info.append(pd.DataFrame(flights.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info = tab_info.append(pd.DataFrame(flights.isnull().sum()/flights.shape[0]*100).T.rename(index={0:'null values (%)'}))
tab_info

## 2. Data Cleaning & Handling Missing Values

Remove columns with excessive missingness, drop rows with critical missing values, and handle remaining missing data.

In [ ]:
# Only keep January for faster EDA (as in tutorial)
flights = flights[flights['MONTH'] == 1]

# Convert date columns
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH','DAY']])

# Remove columns not needed for EDA
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'MONTH','DAY','DAY_OF_WEEK','DATE', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME']
flights.drop(variables_to_remove, axis = 1, inplace = True, errors='ignore')

# Drop rows with missing essential values
essential = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY']
flights.dropna(subset=essential, inplace=True)

# Fill remaining missing numeric values with median
for col in flights.select_dtypes(include=[np.number]).columns:
    flights[col] = flights[col].fillna(flights[col].median())

# Final completeness check
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

## 3. Feature Engineering

Convert scheduled departure to hour, create date and weekend features, and select relevant columns.

In [ ]:
# Helper to convert HHMM to hour
def sched_hour(x):
    x = str(int(x)).zfill(4)
    return int(x[:2])
flights['SCHED_DEP_HOUR'] = flights['SCHEDULED_DEPARTURE'].apply(sched_hour)
flights['IS_WEEKEND'] = pd.to_datetime(flights['DATE']).dt.dayofweek >= 5

## 4. Data Integrity & Consistency

Check for duplicates, negative values, and impossible values.

In [ ]:
flights = flights.drop_duplicates()
print("Negative distances:", (flights['DISTANCE'] < 0).sum())
print("Negative departure delays:", (flights['DEPARTURE_DELAY'] < 0).sum())

## 5. Summary Statistics

Show summary statistics and initial insights.

In [ ]:
flights.describe(include='all')

## 6. Visualizing Airports

Plot the locations of airports and number of flights per airport.

In [ ]:
count_flights = flights['ORIGIN_AIRPORT'].value_counts()
plt.figure(figsize=(11,11))
colors = ['yellow', 'red', 'lightblue', 'purple', 'green', 'orange']
size_limits = [1, 100, 1000, 10000, 100000, 1000000]
labels = []
for i in range(len(size_limits)-1):
    labels.append("{} <.< {}".format(size_limits[i], size_limits[i+1])) 
map = Basemap(resolution='i',llcrnrlon=-180, urcrnrlon=-50,
              llcrnrlat=10, urcrnrlat=75, lat_0=0, lon_0=0,)
map.shadedrelief()
map.drawcoastlines()
map.drawcountries(linewidth = 3)
map.drawstates(color='0.3')
for index, (code, y,x) in airports[['IATA_CODE', 'LATITUDE', 'LONGITUDE']].iterrows():
    if code not in count_flights: continue
    x, y = map(x, y)
    isize = [i for i, val in enumerate(size_limits) if val < count_flights[code]]
    ind = isize[-1]
    map.plot(x, y, marker='o', markersize = ind+5, markeredgewidth = 1, color = colors[ind],
             markeredgecolor='k', label = labels[ind])
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
key_order = ('1 <.< 100', '100 <.< 1000', '1000 <.< 10000',
             '10000 <.< 100000', '100000 <.< 1000000')
new_label = OrderedDict()
for key in key_order:
    if key in by_label:
        new_label[key] = by_label[key]
plt.legend(new_label.values(), new_label.keys(), loc = 1, prop= {'size':11},
           title='Number of flights per year', frameon = True, framealpha = 1)
plt.show()

## 7. Airline Delay Analysis

Visualize airline statistics and delay distributions.

In [ ]:
airlines_names = pd.read_csv('airlines.csv')
abbr_companies = airlines_names.set_index('IATA_CODE')['AIRLINE'].to_dict()
df2 = flights.loc[:, ['AIRLINE', 'DEPARTURE_DELAY']]
df2['AIRLINE'] = df2['AIRLINE'].replace(abbr_companies)
plt.figure(figsize=(12,5))
sns.boxplot(x='AIRLINE', y='DEPARTURE_DELAY', data=df2)
plt.title('Departure Delay Distribution by Airline')
plt.xticks(rotation=45)
plt.show()

In [ ]:
delay_type = lambda x:((0,1)[x > 5],2)[x > 45]
flights['DELAY_LEVEL'] = flights['DEPARTURE_DELAY'].apply(delay_type)
plt.figure(figsize=(10,6))
sns.countplot(y="AIRLINE", hue='DELAY_LEVEL', data=flights)
plt.title('Delay Levels by Airline')
plt.show()

## 8. Delay Distribution by Airline

Show normalized distribution of delays for each airline.

In [ ]:
def func(x, a, b):
    return a * np.exp(-x/b)
points = [] ; label_company = []
fig = plt.figure(1, figsize=(11,11))
i = 0
for carrier_name in [abbr_companies[x] for x in flights['AIRLINE'].unique() if x in abbr_companies]:
    i += 1
    ax = fig.add_subplot(5,3,i)    
    n, bins, patches = plt.hist(x = df2[df2['AIRLINE']==carrier_name]['DEPARTURE_DELAY'],
                                range = (15,180), density=True, bins= 60)
    bin_centers = bins[:-1] + 0.5 * (bins[1:] - bins[:-1])    
    if len(bin_centers) > 0 and len(n) > 0:
        try:
            popt, pcov = scipy.optimize.curve_fit(func, bin_centers, n, p0 = [1, 2])
            points.append(popt)
            label_company.append(carrier_name)
            plt.plot(bin_centers, func(bin_centers, *popt), 'r-', linewidth=3)
        except:
            pass
    plt.title(carrier_name, fontsize = 10)
plt.tight_layout()

## 9. Departure vs Arrival Delays

Compare mean delays at departure and arrival for each airline.

In [ ]:
plt.figure(figsize=(11,6))
sns.barplot(x="DEPARTURE_DELAY", y="AIRLINE", data=flights, color="lightskyblue", ci=None)
sns.barplot(x="ARRIVAL_DELAY", y="AIRLINE", data=flights, color="r", hatch = '///', alpha = 0.0, ci=None)
plt.xlabel('Mean delay [min] (@departure: blue, @arrival: hatch lines)', fontsize=14)
plt.show()

## 10. Airport Analysis

Show number of airports, mean delays by airport, and heatmap for a subset.

In [ ]:
airport_mean_delays = pd.DataFrame(pd.Series(flights['ORIGIN_AIRPORT'].unique()))
airport_mean_delays.set_index(0, drop = True, inplace = True)
for carrier in abbr_companies.keys():
    df1 = flights[flights['AIRLINE'] == carrier]
    test = df1['DEPARTURE_DELAY'].groupby(flights['ORIGIN_AIRPORT']).mean()
    airport_mean_delays[carrier] = test
subset = airport_mean_delays.iloc[:50,:].rename(columns = abbr_companies)
plt.figure(figsize=(12,8))
sns.heatmap(subset, cmap="Accent", vmin=0, vmax=35)
plt.title('Mean Departure Delay by Airport (subset)')
plt.show()

## 11. Temporal Analysis

Analyze delay trends by scheduled hour.

In [ ]:
hourly_delay = flights.groupby('SCHED_DEP_HOUR')['DEPARTURE_DELAY'].mean()
plt.figure(figsize=(10,4))
sns.lineplot(x=hourly_delay.index, y=hourly_delay.values)
plt.title('Average Departure Delay by Scheduled Hour')
plt.xlabel('Scheduled Departure Hour')
plt.ylabel('Average Delay (min)')
plt.show()

## 12. Outlier Handling

Visualize and cap outliers in delay columns.

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(flights['DEPARTURE_DELAY'], bins=100, kde=True)
plt.xlim(-20, 200)
plt.title('Histogram of Departure Delays')
plt.show()
delay_cap = flights['DEPARTURE_DELAY'].mean() + 3*flights['DEPARTURE_DELAY'].std()
flights['DEPARTURE_DELAY_CAPPED'] = np.where(flights['DEPARTURE_DELAY'] > delay_cap, delay_cap, flights['DEPARTURE_DELAY'])

## 13. Initial Visual Representation of Key Findings

Show key findings with simple plots.

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='IS_WEEKEND', y='DEPARTURE_DELAY_CAPPED', data=flights)
plt.title('Departure Delay by Weekend/Weekday')
plt.xticks([0,1], ['Weekday', 'Weekend'])
plt.show()

In [ ]:
top_airports = flights.groupby('ORIGIN_AIRPORT')['DEPARTURE_DELAY_CAPPED'].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,4))
sns.barplot(x=top_airports.index, y=top_airports.values)
plt.title('Top 10 Origin Airports by Average Departure Delay')
plt.xlabel('Origin Airport')
plt.ylabel('Avg Departure Delay (min)')
plt.show()

## 14. Summary

- Data cleaned, missing values handled, features engineered.
- Outliers capped for robust analysis.
- Patterns: Delays increase later in the day, vary by airline and airport.
- Initial visualizations highlight key trends for further modeling.